In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import create_engine, func
from sqlalchemy import create_engine, inspect

In [2]:
phytoplankton = pd.read_csv("data/phytoplankton.csv")
zooplankton =  pd.read_csv("data/zooplankton.csv")
phytoplankton_color = pd.read_csv("data/phytoplankton_ci.csv")

In [3]:

phytoplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,caab_code,taxon_start_date,taxon_per_m3,phyto_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880720,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,50566000.0,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880722,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030054.0,NaN,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880721,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030900.0,2011-05-17,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,2011-09-20,Tripos spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880726,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,99530018.0,2011-05-17,6666.7,NaN,Wallenius Wilhelmsen,2011-09-15,Pseudo-nitzschia spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (134.91247 -37.87656)
4,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880727,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,NaN,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (134.91247 -37.87656)


In [4]:
#rounding coordinates
phytoplankton = phytoplankton.round(2)
phytoplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,caab_code,taxon_start_date,taxon_per_m3,phyto_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880720,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,50566000.0,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880722,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030054.0,NaN,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880721,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030900.0,2011-05-17,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,2011-09-20,Tripos spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880726,ABAB,-37.88,134.91,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,99530018.0,2011-05-17,6666.7,NaN,Wallenius Wilhelmsen,2011-09-15,Pseudo-nitzschia spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (134.91247 -37.87656)
4,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880727,ABAB,-37.88,134.91,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,NaN,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (134.91247 -37.87656)


In [5]:
#putting lat and long into one column

#phytoplankton["coordinates"] = "[" + phytoplankton["Latitude"].astype(str) +", " + phytoplankton["Longitude"].astype(str) + "]"

In [6]:
#drop unneeded columns for phytoplankton APIs
phyto = phytoplankton.drop(["id",
                                    "Time_24hr",
                                    "FID",
                                    "SampleDateUTC",
#                                    "Latitude",
#                                    "Longitude",
                                    "Month",
                                    "Day",
                                    "family",
                                    "genus",
#                                    "taxon_group",
                                    "species",
                                    "caab_code",
                                   "taxon_start_date",
                                    "taxon_name",
                                   "phyto_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "geom"
                                  ],
                                  axis=1)

In [7]:
phyto.head()

,Route,Latitude,Longitude,Year,taxon_group,taxon_per_m3
0,ABAB,-37.91,135.33,2020,Ciliate,3333.3
1,ABAB,-37.91,135.33,2020,Dinoflagellate,666.7
2,ABAB,-37.91,135.33,2020,Dinoflagellate,666.7
3,ABAB,-37.88,134.91,2020,Pennate diatom,6666.7
4,ABAB,-37.88,134.91,2020,Dinoflagellate,3333.3


# phyto_taxon_group API

In [8]:
phyto_taxon_group = phyto.drop(["Latitude", "Longitude"], axis=1)

phyto_taxon_group.head()

,Route,Year,taxon_group,taxon_per_m3
0,ABAB,2020,Ciliate,3333.3
1,ABAB,2020,Dinoflagellate,666.7
2,ABAB,2020,Dinoflagellate,666.7
3,ABAB,2020,Pennate diatom,6666.7
4,ABAB,2020,Dinoflagellate,3333.3


# phyto_routes API

In [9]:
# Routes API
# reset index and and a log of taxon_main_routes for more informative heatmap
phyto_routes = phyto.drop(["taxon_group"], axis=1)

phyto_routes = phyto_routes.reset_index().drop(["index"],axis=1)
phyto_routes["log2_m3"] = np.log2(phyto_routes["taxon_per_m3"]).round(1)
phyto_routes.head()

,Route,Latitude,Longitude,Year,taxon_per_m3,log2_m3
0,ABAB,-37.91,135.33,2020,3333.3,11.7
1,ABAB,-37.91,135.33,2020,666.7,9.4
2,ABAB,-37.91,135.33,2020,666.7,9.4
3,ABAB,-37.88,134.91,2020,6666.7,12.7
4,ABAB,-37.88,134.91,2020,3333.3,11.7


# phyto_groupby_route

In [10]:
# find the average taxon per m3 grouped by each route and year and turn it into a dataframe
phyto_groupby_route = pd.DataFrame(phyto_routes.groupby(by=["Route","Year"])["taxon_per_m3"].mean())

phyto_groupby_route.head()

taxon_per_m3
Route Year              
ABAB  2020        3333.3
ABFR  2014        3333.3
      2015        3333.3
      2016        3333.3
      2017         666.7

In [11]:
# get list of unique routes to unimportant ones for per year sample consistency/data limit
column_values = phytoplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

['ABAB' 'ABFR' 'ADAB' 'ADME' 'ANHO' 'AUHO' 'BRER' 'BRFI' 'BRGL' 'BRNC'
 'BRPK' 'BRSY' 'BUNE' 'CAGL' 'EDMI' 'FRAN' 'FRBO' 'FRSY' 'GLTO' 'HOAN'
 'HOFR' 'HOSO' 'HOSY' 'IIOE' 'MEAB' 'MEAD' 'MEFR' 'MEPK' 'NCME' 'NEBU'
 'PKME' 'SOHO' 'SYBR' 'SYHO' 'SYME' 'TINT' 'TOCA']


In [12]:
#total count of unique values

item_counts = phytoplankton["Route"].value_counts()
print(item_counts)

HOAN    15806
ANHO     6284
BRSY     3470
BRPK     2647
SYME     2098
PKME     1952
MEAD     1777
CAGL     1084
ADAB      969
ABFR      743
HOFR      735
MEAB      630
FRBO      554
GLTO      432
HOSY      406
FRAN      390
BRNC      364
BRGL      335
IIOE      315
BRER      299
SYBR      199
BUNE      191
MEFR      190
NCME      182
FRSY      172
SYHO      160
AUHO      147
TINT      144
HOSO      114
EDMI      105
NEBU       98
ABAB       90
MEPK       83
SOHO       55
BRFI       55
TOCA       49
ADME       33
Name: Route, dtype: int64


In [13]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = phytoplankton.loc[phytoplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

ABAB
2020    90
Name: Year, dtype: int64
***************************************************
ABFR
2014    300
2016    220
2017    136
2015     87
Name: Year, dtype: int64
***************************************************
ADAB
2014    488
2016    330
2017     92
2015     59
Name: Year, dtype: int64
***************************************************
ADME
2019    33
Name: Year, dtype: int64
***************************************************
ANHO
2010    1432
2009    1327
2008    1148
2018    1132
2011     702
2013     336
2017     103
2016      79
2019      24
2007       1
Name: Year, dtype: int64
***************************************************
AUHO
2011    147
Name: Year, dtype: int64
***************************************************
BRER
2014    299
Name: Year, dtype: int64
***************************************************
BRFI
2012    55
Name: Year, dtype: int64
***************************************************
BRGL
2014    281
2015     54
Name: Year, dtype: int64
*******

In [14]:
#have only four main routes for sample size due to data limits

phyto_main_4 = phyto_routes.loc[((phyto_routes['Route'] == "ANHO") |
        (phyto_routes["Route"] == 'BRSY')|
        (phyto_routes["Route"] == 'HOAN')|
#        (phyto_routes["Route"] == 'PKME')|
        (phyto_routes["Route"] == 'MEAD')), :]

# phyto_years API 

### Use when  x = Year NOT Route

In [15]:
#phyto_main_4 = phyto_main_routes.drop(["index"],axis=1
phyto_years = phyto_main_4.reset_index().drop(["index", "log2_m3"],axis=1)
phyto_years.head()

,Route,Latitude,Longitude,Year,taxon_per_m3
0,ANHO,-60.10,89.61,2007,666.7
1,ANHO,-63.27,139.89,2008,3333.3
2,ANHO,-63.27,139.89,2008,10000.0
3,ANHO,-63.27,139.89,2008,3333.3
4,ANHO,-63.27,139.89,2008,6666.7


In [16]:
phyto_routes.count


<bound method DataFrame.count of       Route  Latitude  Longitude  Year  taxon_per_m3  log2_m3
0      ABAB    -37.91     135.33  2020        3333.3     11.7
1      ABAB    -37.91     135.33  2020         666.7      9.4
2      ABAB    -37.91     135.33  2020         666.7      9.4
3      ABAB    -37.88     134.91  2020        6666.7     12.7
4      ABAB    -37.88     134.91  2020        3333.3     11.7
...     ...       ...        ...   ...           ...      ...
43352  TOCA    -16.80     145.92  2013         666.7      9.4
43353  TOCA    -16.80     145.92  2013         666.7      9.4
43354  TOCA    -16.80     145.92  2013         666.7      9.4
43355  TOCA    -16.80     145.92  2013         666.7      9.4
43356  TOCA    -16.80     145.92  2013         666.7      9.4

[43357 rows x 6 columns]>

# Zooplankton

In [17]:
#rounding coordinates
zooplankton = zooplankton.round(2)
zooplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,aphiaid,taxon_start_date,taxon_per_m3,zoop_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974299,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,1100.0,2008-09-29,2.00,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974379,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,2081.0,2008-09-29,0.67,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974380,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,1410.0,2013-03-22,1.33,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974378,ABAB,-37.91,135.33,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,106485.0,2008-09-29,4.00,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
4,soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...,60974296,ABAB,-37.88,134.91,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,106652.0,2011-07-08,0.67,NaN,Wallenius Wilhelmsen,2010-11-18,Oithona Grp3 (pointy head) f,External training Dave McKinnon,Identified when plumes or setae on gential seg...,POINT (134.91247 -37.87656)


In [18]:
#coordinates column
#zooplankton["coordinates"] = "[" + zooplankton["Latitude"].astype(str) +", " + zooplankton["Longitude"].astype(str) + "]"

In [19]:
for col in zooplankton.columns:
    print(col)

FID
id
Route
Latitude
Longitude
SampleDateUTC
Year
Month
Day
Time_24hr
taxon_name
family
genus
species
sex
life_stage
taxon_group
taxon_eco_group
caab_code
aphiaid
taxon_start_date
taxon_per_m3
zoop_comments
acknowledgements
first_occurrence
parent_taxon_name
training
comments
geom


In [20]:
#drop unneeded columns for zooplankton
zoo = zooplankton.drop(["id",
                                "FID",
                                "Time_24hr",
                                "SampleDateUTC",
#                                "Latitude",
#                                "Longitude",
                                "family",
                                "Month",
                                "Day",
                                "genus",
#                                "taxon_group",
                                "species",
                                "sex",
                                  "life_stage",
                                    "caab_code",
                                    "aphiaid",  
                                   "taxon_start_date",
                                   "zoop_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                             "taxon_name",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "taxon_eco_group",
                                "geom"
                                  ],
                                  axis=1)

In [21]:
zoo.head()

,Route,Latitude,Longitude,Year,taxon_group,taxon_per_m3
0,ABAB,-37.91,135.33,2020,COPEPOD,2.00
1,ABAB,-37.91,135.33,2020,NON-COPEPOD,0.67
2,ABAB,-37.91,135.33,2020,NON-COPEPOD,1.33
3,ABAB,-37.91,135.33,2020,COPEPOD,4.00
4,ABAB,-37.88,134.91,2020,COPEPOD,0.67


# zoo_taxon_group API

In [22]:
zoo_taxon_group = zoo.drop(["Latitude", "Longitude"], axis=1)
zoo_taxon_group.head()

,Route,Year,taxon_group,taxon_per_m3
0,ABAB,2020,COPEPOD,2.00
1,ABAB,2020,NON-COPEPOD,0.67
2,ABAB,2020,NON-COPEPOD,1.33
3,ABAB,2020,COPEPOD,4.00
4,ABAB,2020,COPEPOD,0.67


# zoo_routes API

In [23]:
#reset add column for logarithmic scale
zoo_routes = zoo.drop(["taxon_group"], axis =1)
zoo_routes = zoo_routes.reset_index().drop(["index"],axis=1)
zoo_routes["log2_m3"] = np.log2(zoo_routes["taxon_per_m3"]).round(1)
zoo_routes.head()

,Route,Latitude,Longitude,Year,taxon_per_m3,log2_m3
0,ABAB,-37.91,135.33,2020,2.00,1.0
1,ABAB,-37.91,135.33,2020,0.67,-0.6
2,ABAB,-37.91,135.33,2020,1.33,0.4
3,ABAB,-37.91,135.33,2020,4.00,2.0
4,ABAB,-37.88,134.91,2020,0.67,-0.6


# zoo_groupby_route API

In [24]:
# find the average taxon per m3 grouped by each route and year and turn it into a dataframe
zoo_groupby_route = pd.DataFrame(zoo_routes.groupby(by=["Route","Year"])["taxon_per_m3"].mean())

zoo_groupby_route.head()

taxon_per_m3
Route Year              
ABAB  2020          1.33
ABFR  2014          0.67
      2015          0.67
      2016          1.33
      2017          1.33

In [25]:
zooplankton.count

<bound method DataFrame.count of                                                       FID        id Route  \
0       soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...  60974299  ABAB   
1       soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...  60974379  ABAB   
2       soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...  60974380  ABAB   
3       soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...  60974378  ABAB   
4       soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a6...  60974296  ABAB   
...                                                   ...       ...   ...   
111409  soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a7...  60930303  TOCA   
111410  soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a7...  60930302  TOCA   
111411  soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a7...  60930205  TOCA   
111412  soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a7...  60930206  TOCA   
111413  soop_auscpr_zoop_flat_map.fid--620ea8f0_17a9a7...  60930318  TOCA   

        Latitude  Longitude         Sample

In [26]:
column_values = zooplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

['ABAB' 'ABFR' 'ADAB' 'ADME' 'ANHO' 'AUHO' 'BRER' 'BRFI' 'BRGL' 'BRNC'
 'BRPK' 'BRSY' 'BUNE' 'CAGL' 'FRBO' 'FRSY' 'GLTO' 'HOAN' 'HOFR' 'HOSO'
 'HOSY' 'IIOE' 'MEAB' 'MEAD' 'MEFR' 'MEPK' 'NCME' 'NEBU' 'PKME' 'SOHO'
 'SYBR' 'SYHO' 'SYME' 'TINT' 'TOCA']


In [27]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = zooplankton.loc[zooplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

ABAB
2020    297
Name: Year, dtype: int64
***************************************************
ABFR
2016    800
2015    661
2017    537
2014    433
Name: Year, dtype: int64
***************************************************
ADAB
2014    1390
2016     593
2017     303
2015     216
Name: Year, dtype: int64
***************************************************
ADME
2019    355
Name: Year, dtype: int64
***************************************************
ANHO
2007    2343
2010    2193
2013    2025
2009    1211
2008     918
2011     867
2018     293
2016     263
2017     192
2019      64
Name: Year, dtype: int64
***************************************************
AUHO
2011    982
Name: Year, dtype: int64
***************************************************
BRER
2014    766
Name: Year, dtype: int64
***************************************************
BRFI
2012    234
Name: Year, dtype: int64
***************************************************
BRGL
2014    1027
2015     265
Name: Year, dtype: int6

In [28]:
#ANHO, BRSY, HOAN, MEAD, PKME

In [29]:
zoo_main_4 = zoo_routes.loc[((zoo_routes['Route'] == "ANHO") |
        (zoo_routes["Route"] == 'BRSY')|
        (zoo_routes["Route"] == 'HOAN')|
#        (zoo_routes["Route"] == 'PKME')|
        (zoo_routes["Route"] == 'MEAD')), :]

# zoo_years API

In [30]:
zoo_years = zoo_main_4.reset_index().drop(["index", "log2_m3"], axis=1)
zoo_years.head()

,Route,Latitude,Longitude,Year,taxon_per_m3
0,ANHO,-60.1,89.61,2007,2.00
1,ANHO,-60.1,89.61,2007,16.67
2,ANHO,-60.1,89.61,2007,0.67
3,ANHO,-60.1,89.61,2007,3.33
4,ANHO,-60.1,89.61,2007,2.67


In [31]:
print(np.log2(zoo_routes["taxon_per_m3"].max()).round(1))


13.5


In [32]:
print(np.log2(phyto_routes["taxon_per_m3"].max()).round(1))

18.3


In [33]:
# # save to csv, remember to delete index in csv to save space
# zoo_main_routes.to_csv("clean_data/zooplankton_clean.csv")
# phyto_main_routes.to_csv("clean_data/phytoplankton_clean.csv")

In [34]:
# for col in phytoplankton_color.columns:
#     print(col)

In [35]:
# #drop unneeded columns for phytoplankton color index
# phytoplankton_color = phytoplankton_color.drop(["FID",
#                                                 "start_port",
#                                                 "end_port",
#                                                 "trip_code",
#                                                 "colour_index",
#                                                 "measurement",
#                                                "acknowledgements",
#                                                "route_frequency",
#                                                "route_start_date",
#                                                "pci_comments",
#                                                 "geom",
#                                                   ],
#                                                   axis=1)

In [36]:
phytoplankton_color

,FID,measurement,route_code,start_port,end_port,route_frequency,route_start_date,vessel_name,trip_code,colour_index,colour,pci_comments,acknowledgements,TIME,geom,latitude,longitude
0,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24321258,ABAB,Great Australian Bight,Great Australian Bight,NaN,NaN,Salome IMO,ABAB20200227,1,VERY_PALE_GREEN,NaN,Wallenius Wilhelmsen,2020-02-27T02:01:35Z,POINT (135.33099 -37.91378),-37.9138,135.3310
1,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24321259,ABAB,Great Australian Bight,Great Australian Bight,NaN,NaN,Salome IMO,ABAB20200227,1,VERY_PALE_GREEN,NaN,Wallenius Wilhelmsen,2020-02-27T02:18:47Z,POINT (135.22632 -37.90462),-37.9046,135.2263
2,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24321260,ABAB,Great Australian Bight,Great Australian Bight,NaN,NaN,Salome IMO,ABAB20200227,1,VERY_PALE_GREEN,NaN,Wallenius Wilhelmsen,2020-02-27T02:35:58Z,POINT (135.12167 -37.89536),-37.8954,135.1217
3,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24321261,ABAB,Great Australian Bight,Great Australian Bight,NaN,NaN,Salome IMO,ABAB20200227,1,VERY_PALE_GREEN,NaN,Wallenius Wilhelmsen,2020-02-27T02:53:10Z,POINT (135.01706 -37.886),-37.8860,135.0171
4,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24321262,ABAB,Great Australian Bight,Great Australian Bight,NaN,NaN,Salome IMO,ABAB20200227,1,VERY_PALE_GREEN,NaN,Wallenius Wilhelmsen,2020-02-27T03:10:21Z,POINT (134.91247 -37.87656),-37.8766,134.9125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27627,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24318877,TOCA,Townsville,Cairns,NaN,NaN,Kweichow IMO 9070694,TOCA20161017,1,VERY_PALE_GREEN,NaN,Swires,2016-10-17T18:52:05Z,POINT (146.14549 -17.27991),-17.2799,146.1455
27628,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24318878,TOCA,Townsville,Cairns,NaN,NaN,Kweichow IMO 9070694,TOCA20161017,1,VERY_PALE_GREEN,NaN,Swires,2016-10-17T19:18:59Z,POINT (146.12171 -17.19942),-17.1994,146.1217
27629,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24318879,TOCA,Townsville,Cairns,NaN,NaN,Kweichow IMO 9070694,TOCA20161017,1,VERY_PALE_GREEN,NaN,Swires,2016-10-17T19:45:53Z,POINT (146.09794 -17.11893),-17.1189,146.0979
27630,soop_auscpr_pci_trajectory_map.fid--404dafa0_1...,24318880,TOCA,Townsville,Cairns,NaN,NaN,Kweichow IMO 9070694,TOCA20161017,1,VERY_PALE_GREEN,NaN,Swires,2016-10-17T20:12:47Z,POINT (146.0742 -17.03844),-17.0384,146.0742


In [37]:
# add csv to data base function
# def read_sql(data):
#     data.to_sql(f"'{data}'", conn, if_exists = "append")


In [38]:
# # add csv to data base
# read_sql(phytoplankton)
# read_sql(zooplankton)
# read_sql(phytoplankton_color)

In [39]:
# Create an engine for the plankton.sqlite database
engine = create_engine("sqlite:///planktontestmain2.sqlite", echo=False)
conn = engine.connect()

# phyto_routes
# phyto_taxon_group
# phyto_years
# phyto_groupby_route

# zoo_routes
# zoo_taxon_group
# zoo_years
# zoo_groupby_route

In [40]:
phyto_routes.head()

,Route,Latitude,Longitude,Year,taxon_per_m3,log2_m3
0,ABAB,-37.91,135.33,2020,3333.3,11.7
1,ABAB,-37.91,135.33,2020,666.7,9.4
2,ABAB,-37.91,135.33,2020,666.7,9.4
3,ABAB,-37.88,134.91,2020,6666.7,12.7
4,ABAB,-37.88,134.91,2020,3333.3,11.7


In [41]:
phyto_routes.to_sql("phytoplankton", conn,)
phyto_taxon_group.to_sql("phytoplankton_taxon_group", conn,)
phyto_years.to_sql("phytoplankton_years", conn,)
zoo_routes.to_sql("zooplankton", conn,)
zoo_taxon_group.to_sql("zooplankton_taxon_group", conn,)
zoo_years.to_sql("zooplankton_years", conn,)
zoo_groupby_route.to_sql("zooplankton_groupby_route", conn,)
phyto_groupby_route.to_sql("phytoplankton_groupby_route", conn,)
#phytoplankton_color.to_sql("phytoplankton_color_index", conn,)

In [42]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [43]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [44]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [45]:
# Use `engine.execute` to select and display the first 10 rows from the station table
engine.execute('SELECT * FROM zooplankton LIMIT 5').fetchall()

[(0, 'ABAB', -37.91, 135.33, 2020, 2.0, 1.0),
 (1, 'ABAB', -37.91, 135.33, 2020, 0.67, -0.6),
 (2, 'ABAB', -37.91, 135.33, 2020, 1.33, 0.4),
 (3, 'ABAB', -37.91, 135.33, 2020, 4.0, 2.0),
 (4, 'ABAB', -37.88, 134.91, 2020, 0.67, -0.6)]

In [46]:
# Query phytoplankton Records in the the Database
data = pd.read_sql("SELECT * FROM phytoplankton", conn)
dataZ = pd.read_sql("SELECT * FROM zooplankton", conn)

In [47]:
data.head()

,index,Route,Latitude,Longitude,Year,taxon_per_m3,log2_m3
0,0,ABAB,-37.91,135.33,2020,3333.3,11.7
1,1,ABAB,-37.91,135.33,2020,666.7,9.4
2,2,ABAB,-37.91,135.33,2020,666.7,9.4
3,3,ABAB,-37.88,134.91,2020,6666.7,12.7
4,4,ABAB,-37.88,134.91,2020,3333.3,11.7


In [48]:
dataZ.head()

,index,Route,Latitude,Longitude,Year,taxon_per_m3,log2_m3
0,0,ABAB,-37.91,135.33,2020,2.00,1.0
1,1,ABAB,-37.91,135.33,2020,0.67,-0.6
2,2,ABAB,-37.91,135.33,2020,1.33,0.4
3,3,ABAB,-37.91,135.33,2020,4.00,2.0
4,4,ABAB,-37.88,134.91,2020,0.67,-0.6


In [49]:
zoo_years = pd.read_sql("SELECT * FROM phytoplankton_years WHERE Year > 2016", conn)
zoo_years.head()

,index,Route,Latitude,Longitude,Year,taxon_per_m3
0,5025,ANHO,-46.69,142.17,2017,3333.3
1,5026,ANHO,-46.69,142.17,2017,6666.7
2,5027,ANHO,-46.69,142.17,2017,3333.3
3,5028,ANHO,-46.48,142.54,2017,10000.0
4,5029,ANHO,-46.48,142.54,2017,3333.3
